In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from operator import itemgetter
import random

In [2]:
Pdata= pd.read_csv('AuctionData.csv')
PositionCTR=pd.read_csv("CTR10.csv")

In [28]:
QualityScore=pd.read_csv("Keyword6\QualityScore.csv")
ClickRate=pd.read_csv("Keyword6\Clickrate.csv")
BidData=pd.read_csv("Keyword6\BidArray.csv")

In [29]:
BidData

,BidderID,Price
0,ae30807c-5316-42ee-a031-589fdb08e4ba,28050.0
1,52852966-92dd-43ac-8435-01b9e6021c7c,10000.0
2,6524d297-40d9-4a96-90c0-0d5aa0c61da6,4890.0
3,54bff083-a345-436d-99f4-3e70f8969567,3750.0
4,a2860a55-f8a1-4d62-8edc-49fb88661adc,2000.0
5,ebd4f3cf-2d58-41d9-a848-e3db688ba491,2000.0
6,b5f02143-b63e-4062-bb82-0624909e55ac,1500.0
7,9eb5a22f-1870-42b1-aaae-2faa5235ae34,1000.0
8,223f15fb-45b3-4a53-bdcf-c173a6e8c38a,850.0
9,10fa3930-fa02-4c4a-8fd6-4bc5d831c171,600.0


In [30]:
# for i in range(2,7):
#     path="Keyword%d\BidArray.csv"%i
#     BidData=pd.read_csv(path)
#     for j in range(20):
#         for k in range(j+1,20):
#             if BidData['Price'][k]>BidData['Price'][j]:
#                 TempN=BidData['BidderID'][k]
#                 TempB=BidData['Price'][k]
#                 BidData['BidderID'][k]=BidData['BidderID'][j]
#                 BidData['Price'][k]=BidData['Price'][j]
#                 BidData['Price'][j]=TempB
#                 BidData['BidderID'][j]=TempN
#         BidData.to_csv(path,index=False,sep=',')

In [5]:
# Excute Dynamically Adjusting Bid Strategy
def DAB(BidList,PositionImp,ClickRate,TrueValue):
    NewBid=np.zeros(20)
    for i in range(20):
        for m in range(len(ClickRate)):
            if BidList[i][0]==ClickRate['Bidder'][m]:
                CR=ClickRate['ClickRate'][m]
                break
        TrueV=TrueValue['Price'][i]
#         if i==0:
#             print(CR)
        Bid=np.empty((20,2),dtype='object')
        for j in range(20):
            if j!=i:
                Bid[j][0]=BidList[j][0]
                Bid[j][1]=BidList[j][1]
            if j==i:
                Bid[j][0]=BidList[j][0]
                Bid[j][1]=0
                lastBid=BidList[j][1]
        Bid=Bid[np.argsort(-Bid[:,1])]
        for j in range(len(Bid)):
#             if Bid[j][1]==0:
#                 Bid[j][1]=Bid[j][1]+200
#             else:
              Bid[j][1]=Bid[j][1]+random.randint(1,30)
#         if i==5:
#             print(Bid)
        Utility=np.zeros(10)
        for j in range(10):
            Utility[j]=(TrueV-Bid[j][1])*PositionImp['ClickThroughRate'][j]
#         if i==5:
#             print(Utility)
        MaxIndex=0
        MaxR=Utility[0]
        for j in range(1,10):
            if Utility[j]>=MaxR:
                MaxIndex=j
                MaxR=Utility[j]
        if MaxR<=0:
            NewBid[i]=TrueV*0.8
        else:
            NewBid[i]=Bid[MaxIndex][1]
    return NewBid

# Generate GFP Bids in DAB

In [31]:
BidMatrixGFP=np.empty((20,21),dtype='object')
for i in range(20):
    BidMatrixGFP[i][0]=BidData['BidderID'][i]
    BidMatrixGFP[i][1]=BidData['Price'][i]*0.8
LastBid=np.empty((20,2),dtype='object')
for i in range(20):
    LastBid[i][0]=BidMatrixGFP[i][0]
    LastBid[i][1]=BidMatrixGFP[i][1]
for j in range(19):
    NewBid=DAB(LastBid,PositionCTR,ClickRate,BidData)
    for i in range(20):
        BidMatrixGFP[i][2+j]=NewBid[i]
    for i in range(len(LastBid)):
        LastBid[i][1]=NewBid[i]

In [32]:
np.savetxt("Keyword6\BidMatrixGFP.csv", BidMatrixGFP ,fmt='%5s',delimiter=",")

# Generate GSP Bids in LEF

In [33]:
GSPBidk1=np.empty((20,2),dtype='object')
for i in range(20):
    GSPBidk1[i][0]=BidData['BidderID'][i]
C=np.zeros(10)
for i in range(9):
    C[i]=PositionCTR['ClickThroughRate'][i+1]/PositionCTR['ClickThroughRate'][i]
# Excute Locally-Envy Free Bid Strategy
for i in range(20):
    Sum=0
    for j in range(i,10):
        Cj=1
        for k in range(i,j+1):
            Cj*=C[k-1]
        Sum+=(Cj*(BidData['Price'][j]-BidData['Price'][j+1]))
    print(Sum)
    NewBid=BidData['Price'][i]-Sum
    GSPBidk1[i][1]=NewBid

0.0
3975.346589890997
2105.5085268747616
1818.4231870158733
963.0477144538175
1099.2425564036596
686.2613760316709
342.4197623278269
239.2080923830131
0.0
0
0
0
0
0
0
0
0
0
0


In [34]:
np.savetxt("Keyword6\BidMatrixGSP.csv", GSPBidk1 ,fmt='%5s',delimiter=",")

# End

In [129]:
PositionCTR['Impression'][1]-=2000000
PositionCTR['Impression'][2]-=6000000

In [6]:
BidData

,BidderID,Price
0,24fffcaf-134a-4a99-9756-f9261b74ed6a,6000.0
1,d42271d5-7367-4ffb-95a5-4d1cb2b0dc59,4770.0
2,6f15d700-facb-45be-aaba-bbd1fb359833,2500.0
3,8b105e0d-9c93-499f-b67f-2f2a26407871,1250.0
4,722ba2cc-8354-4029-adf5-9b901b4afb5c,930.0
5,327b7d84-2646-4818-a1ee-a45a2e7d716c,750.0
6,2de3966f-0109-4c7b-b4a1-fd1dde8b3bb4,650.0
7,409c8972-a5fa-4233-8b47-7d3d4e4ba961,540.0
8,6bcbb8b0-002d-43b4-ae73-e270e1318efd,500.0
9,3a100534-7b8b-4536-9e35-73b4ae6b2a59,300.0


In [7]:
QualityScore

,BidderID,Quality
0,3a100534-7b8b-4536-9e35-73b4ae6b2a59,1.361538
1,b4f89140-1764-4d50-889e-95aae2ed8403,0.000000
2,722ba2cc-8354-4029-adf5-9b901b4afb5c,2.694943
3,327b7d84-2646-4818-a1ee-a45a2e7d716c,0.000000
4,fd0b1082-2d68-43c6-849e-6ef7f092c09d,1.845617
5,8b105e0d-9c93-499f-b67f-2f2a26407871,1.740993
6,24fffcaf-134a-4a99-9756-f9261b74ed6a,1.494618
7,ccebcd2f-3eb2-410c-a443-cf8e1fb4fc78,1.190846
8,409c8972-a5fa-4233-8b47-7d3d4e4ba961,1.534917
9,6f15d700-facb-45be-aaba-bbd1fb359833,0.587434


In [13]:
ClickRate

,Bidder,ClickRate
0,3a100534-7b8b-4536-9e35-73b4ae6b2a59,0.008043
1,b4f89140-1764-4d50-889e-95aae2ed8403,0.000000
2,722ba2cc-8354-4029-adf5-9b901b4afb5c,0.500000
3,327b7d84-2646-4818-a1ee-a45a2e7d716c,0.000000
4,fd0b1082-2d68-43c6-849e-6ef7f092c09d,0.008399
5,8b105e0d-9c93-499f-b67f-2f2a26407871,0.005236
6,24fffcaf-134a-4a99-9756-f9261b74ed6a,0.000999
7,ccebcd2f-3eb2-410c-a443-cf8e1fb4fc78,0.004437
8,409c8972-a5fa-4233-8b47-7d3d4e4ba961,0.007042
9,6f15d700-facb-45be-aaba-bbd1fb359833,0.013736


In [8]:
BidMatrixGFP=np.empty((20,21),dtype='object')

In [9]:
for i in range(20):
    BidMatrixGFP[i][0]=BidData['BidderID'][i]
    BidMatrixGFP[i][1]=BidData['Price'][i]*0.8
LastBid=np.empty((20,2),dtype='object')
for i in range(20):
    LastBid[i][0]=BidMatrixGFP[i][0]
    LastBid[i][1]=BidMatrixGFP[i][1]


In [5]:
# Excute Dynamically Adjusting Bid Strategy
def DAB(BidList,PositionImp,ClickRate,TrueValue):
    NewBid=np.zeros(20)
    for i in range(20):
        for m in range(len(ClickRate)):
            if BidList[i][0]==ClickRate['Bidder'][m]:
                CR=ClickRate['ClickRate'][m]
                break
        TrueV=TrueValue['Price'][i]
#         if i==0:
#             print(CR)
        Bid=np.empty((20,2),dtype='object')
        for j in range(20):
            if j!=i:
                Bid[j][0]=BidList[j][0]
                Bid[j][1]=BidList[j][1]
            if j==i:
                Bid[j][0]=BidList[j][0]
                Bid[j][1]=0
                lastBid=BidList[j][1]
        Bid=Bid[np.argsort(-Bid[:,1])]
        for j in range(len(Bid)):
#             if Bid[j][1]==0:
#                 Bid[j][1]=Bid[j][1]+200
#             else:
              Bid[j][1]=Bid[j][1]*1.1
#         if i==5:
#             print(Bid)
        Utility=np.zeros(10)
        for j in range(10):
            Utility[j]=(TrueV-Bid[j][1])*PositionImp['ClickThroughRate'][j]
#         if i==5:
#             print(Utility)
        MaxIndex=0
        MaxR=Utility[0]
        for j in range(1,10):
            if Utility[j]>=MaxR:
                MaxIndex=j
                MaxR=Utility[j]
        if MaxR<=0:
            NewBid[i]=TrueV*0.8
        else:
            NewBid[i]=Bid[MaxIndex][1]
    return NewBid

In [11]:
LastBid=np.empty((20,2),dtype='object')
for i in range(20):
    LastBid[i][0]=BidMatrixGFP[i][0]
    LastBid[i][1]=BidMatrixGFP[i][1]

In [12]:
LastBid

array([['24fffcaf-134a-4a99-9756-f9261b74ed6a', 4800.0],
       ['d42271d5-7367-4ffb-95a5-4d1cb2b0dc59', 3816.0],
       ['6f15d700-facb-45be-aaba-bbd1fb359833', 2000.0],
       ['8b105e0d-9c93-499f-b67f-2f2a26407871', 1000.0],
       ['722ba2cc-8354-4029-adf5-9b901b4afb5c', 744.0],
       ['327b7d84-2646-4818-a1ee-a45a2e7d716c', 600.0],
       ['2de3966f-0109-4c7b-b4a1-fd1dde8b3bb4', 520.0],
       ['409c8972-a5fa-4233-8b47-7d3d4e4ba961', 432.0],
       ['6bcbb8b0-002d-43b4-ae73-e270e1318efd', 400.0],
       ['3a100534-7b8b-4536-9e35-73b4ae6b2a59', 240.0],
       ['dc3506ad-c068-4aaa-b00f-145d816adefb', 240.0],
       ['54061b39-6e18-45d1-93f4-13adb3ea1986', 224.0],
       ['b4f89140-1764-4d50-889e-95aae2ed8403', 216.0],
       ['fd0b1082-2d68-43c6-849e-6ef7f092c09d', 192.0],
       ['af413332-8015-480c-9bc5-132de70f9e1d', 184.0],
       ['e7659286-f17e-4d25-ae1b-7bf7bfe55ab5', 160.0],
       ['ec841a13-dd12-4248-8d11-11345d4cce01', 152.0],
       ['62ac6e40-3734-4c74-bb74-def00962672

In [13]:
NewBid=DAB(LastBid,PositionCTR,ClickRate,BidData)

In [14]:
NewBid

array([2200., 1100., 1100.,  572.,  264.,  264.,  264.,  264.,  264.,
        264.,  264.,  264.,  264.,  192.,  184.,  160.,  152.,  152.,
        120.,   80.])

In [ ]:
for i in range(20):
    BidMatrixGFP[i][2]=NewBid[i]

In [15]:
for j in range(19):
    NewBid=DAB(LastBid,PositionCTR,ClickRate,BidData)
    for i in range(20):
        BidMatrixGFP[i][2+j]=NewBid[i]
    for i in range(len(LastBid)):
        LastBid[i][1]=NewBid[i]

In [18]:
BidMatrixGFP[8]

array(['6bcbb8b0-002d-43b4-ae73-e270e1318efd', 400.0, 264.0,
       290.40000000000003, 319.44000000000005, 351.38400000000007,
       386.5224000000001, 425.1746400000002, 264.0, 290.40000000000003,
       319.44000000000005, 264.0, 290.40000000000003, 319.44000000000005,
       264.0, 290.40000000000003, 319.44000000000005, 264.0,
       290.40000000000003, 319.44000000000005, 264.0], dtype=object)

In [19]:
np.savetxt("BidMatrixGFPk1.csv", BidMatrixGFP ,fmt='%5s',delimiter=",")

In [29]:
PositionCTR

,Impression,Click,ClickThroughRate
0,47111917.0,874080.0,0.018553
1,47412275.0,484640.0,0.010222
2,49105765.0,357238.0,0.007275
3,37681588.0,183735.0,0.004876
4,32541800.0,139014.0,0.004272
5,17732749.0,70195.0,0.003958
6,16145931.0,52066.0,0.003225
7,15237973.0,43231.0,0.002837
8,13933974.0,37825.0,0.002715
9,13313311.0,35940.0,0.002700


In [24]:
BidData

,BidderID,Price
0,24fffcaf-134a-4a99-9756-f9261b74ed6a,6000.0
1,d42271d5-7367-4ffb-95a5-4d1cb2b0dc59,4770.0
2,6f15d700-facb-45be-aaba-bbd1fb359833,2500.0
3,8b105e0d-9c93-499f-b67f-2f2a26407871,1250.0
4,722ba2cc-8354-4029-adf5-9b901b4afb5c,930.0
5,327b7d84-2646-4818-a1ee-a45a2e7d716c,750.0
6,2de3966f-0109-4c7b-b4a1-fd1dde8b3bb4,650.0
7,409c8972-a5fa-4233-8b47-7d3d4e4ba961,540.0
8,6bcbb8b0-002d-43b4-ae73-e270e1318efd,500.0
9,3a100534-7b8b-4536-9e35-73b4ae6b2a59,300.0


In [25]:
GSPBidk1=np.empty((20,2),dtype='object')

In [36]:
for i in range(20):
    GSPBidk1[i][0]=BidData['BidderID'][i]

In [30]:
C=np.zeros(10)

In [47]:
C=np.zeros(10)
for i in range(9):
    C[i]=PositionCTR['ClickThroughRate'][i+1]/PositionCTR['ClickThroughRate'][i]

In [48]:
C

array([0.55094476, 0.71169958, 0.67025109, 0.87610119, 0.92664448,
       0.81463115, 0.87978582, 0.95683237, 0.99446168, 0.        ])

In [49]:
# Excute Locally-Envy Free Bid Strategy
for i in range(20):
    Sum=0
    for j in range(i,10):
        Cj=1
        for k in range(i,j+1):
            Cj*=C[k-1]
        Sum+=(Cj*(BidData['Price'][j]-BidData['Price'][j+1]))
    print(Sum)
    NewBid=BidData['Price'][i]-Sum
    GSPBidk1[i][1]=NewBid

0.0
1942.1564867814575
1255.1383430396659
513.5788834694656
446.248487116555
329.3572420214603
255.42999390836982
203.55294392927726
191.36647390641048
0.0
0
0
0
0
0
0
0
0
0
0


In [50]:
GSPBidk1

array([['24fffcaf-134a-4a99-9756-f9261b74ed6a', 6000.0],
       ['d42271d5-7367-4ffb-95a5-4d1cb2b0dc59', 2827.8435132185423],
       ['6f15d700-facb-45be-aaba-bbd1fb359833', 1244.8616569603341],
       ['8b105e0d-9c93-499f-b67f-2f2a26407871', 736.4211165305344],
       ['722ba2cc-8354-4029-adf5-9b901b4afb5c', 483.751512883445],
       ['327b7d84-2646-4818-a1ee-a45a2e7d716c', 420.6427579785397],
       ['2de3966f-0109-4c7b-b4a1-fd1dde8b3bb4', 394.57000609163015],
       ['409c8972-a5fa-4233-8b47-7d3d4e4ba961', 336.44705607072274],
       ['6bcbb8b0-002d-43b4-ae73-e270e1318efd', 308.6335260935895],
       ['3a100534-7b8b-4536-9e35-73b4ae6b2a59', 300.0],
       ['dc3506ad-c068-4aaa-b00f-145d816adefb', 300.0],
       ['54061b39-6e18-45d1-93f4-13adb3ea1986', 280.0],
       ['b4f89140-1764-4d50-889e-95aae2ed8403', 270.0],
       ['fd0b1082-2d68-43c6-849e-6ef7f092c09d', 240.0],
       ['af413332-8015-480c-9bc5-132de70f9e1d', 230.0],
       ['e7659286-f17e-4d25-ae1b-7bf7bfe55ab5', 200.0],
    

In [51]:
np.savetxt("BidMatrixGSPk1.csv", GSPBidk1 ,fmt='%5s',delimiter=",")